<a href="https://colab.research.google.com/github/AvaneSauhard/HumanActivityRecognition/blob/main/HAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#importing libraries
from google.colab import drive
import numpy as np
import pandas as pd
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
import keras
from tensorflow.keras.utils import to_categorical
from sklearn.utils import resample
import itertools
from sklearn.metrics import classification_report, confusion_matrix
from keras.layers import Input 

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
def load_file(filepath):
  data=pd.read_csv(filepath,header=None,delim_whitespace=True)
  return data.values

In [ ]:
def load_group(filenames,filepaths):
  load=list()
  for name in filenames:
    data=load_file(filepaths+name)
    load.append(data)
  load=np.dstack(load)
  return load


In [ ]:
filepath_train="drive/MyDrive/UCI HAR Dataset/train/Inertial Signals/"
filename_train=list()
filename_train+=['total_acc_x_train.txt','total_acc_y_train.txt','total_acc_z_train.txt']
filename_train+=['body_acc_x_train.txt','body_acc_y_train.txt','body_acc_z_train.txt']
filename_train+=['body_gyro_x_train.txt','body_gyro_y_train.txt','body_gyro_z_train.txt']
train_X=load_group(filename_train,filepath_train)
train_Y=load_file("drive/MyDrive/UCI HAR Dataset/train/y_train.txt")

In [ ]:
print(train_X.shape,train_Y.shape)

(7352, 128, 9) (7352, 1)


In [ ]:
filepath_test="drive/MyDrive/UCI HAR Dataset/test/Inertial Signals/"
filename_test=list()
filename_test+=['total_acc_x_test.txt','total_acc_y_test.txt','total_acc_z_test.txt']
filename_test+=['body_acc_x_test.txt','body_acc_y_test.txt','body_acc_z_test.txt']
filename_test+=['body_gyro_x_test.txt','body_gyro_y_test.txt','body_gyro_z_test.txt']
test_X=load_group(filename_test,filepath_test)
test_Y=load_file("drive/MyDrive/UCI HAR Dataset/test/y_test.txt")

In [ ]:
print(test_X.shape,test_Y.shape)

(2947, 128, 9) (2947, 1)


In [ ]:
train_Y=train_Y-1
test_Y=test_Y-1

In [ ]:
train_Y=to_categorical(train_Y)
test_Y=to_categorical(test_Y)

In [ ]:
print(train_X.shape,train_Y.shape,test_X.shape,test_Y.shape)

(7352, 128, 9) (7352, 6) (2947, 128, 9) (2947, 6)


In [ ]:
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import Dense
from keras.models import Model
from keras.models import Sequential

def build_model():
	verbose, epochs, batch_size = 0, 15, 32
	n_timesteps, n_features, n_outputs = train_X.shape[1], train_X.shape[2], train_Y.shape[1]
	model = Sequential()
	model.add(Conv1D(filters=64, kernel_size=5, activation='relu', input_shape=(n_timesteps,n_features)))
	model.add(Conv1D(filters=64, kernel_size=9, activation='relu'))
	model.add(Dropout(0.5))
	model.add(MaxPooling1D(pool_size=2))
	model.add(Flatten())
	model.add(Dense(100, activation='relu'))
	model.add(Dense(n_outputs, activation='softmax'))
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	model.summary()
	model.fit(train_X, train_Y, epochs=12, batch_size=32)
	accuracy = model.evaluate(test_X, test_Y, batch_size=32)
	return accuracy
  

In [ ]:
model = build_model()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_2 (Conv1D)           (None, 124, 64)           2944      
                                                                 
 conv1d_3 (Conv1D)           (None, 116, 64)           36928     
                                                                 
 dropout_1 (Dropout)         (None, 116, 64)           0         
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 58, 64)           0         
 1D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 3712)              0         
                                                                 
 dense_2 (Dense)             (None, 100)               371300    
                                                      